In [218]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "momory_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

In [219]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    temperature=0.1, # 창의력 정도
    model = "gpt-4.1-mini",
    verbose=True
)

In [220]:
from typing import Dict
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser

In [221]:
from typing import Dict, Optional
from pydantic import BaseModel, Field

class ProfileSummary(BaseModel):
    name: Optional[str] = Field(None, description="사용자 이름")
    age: Optional[int] = Field(None, description="사용자 나이")
    gender: Optional[str] = Field(None, description="사용자 성별", examples=['남', '여'])
    tone: str = "neutral"   # 필요하다면 모델에 포함

profiles: Dict[str, ProfileSummary] = {}

def get_profile(session_id: str) -> ProfileSummary:
    # setdefault의 기본값을 Profile 인스턴스로
    return profiles.setdefault(session_id, ProfileSummary())

def set_profile(session_id: str, profile_data: ProfileSummary) -> None:
    profile = get_profile(session_id)
    update = profile_data.model_dump(exclude_unset=True)

    # None 값은 제외하고 업데이트
    for k, v in update.items():
        if v is not None:
            setattr(profile, k, v)  

profile_parser = PydanticOutputParser(pydantic_object=ProfileSummary)
fmt = profile_parser.get_format_instructions()

In [222]:
from langchain_core.prompts import ChatPromptTemplate

profile_prompt = ChatPromptTemplate.from_messages([
    ("system",  """[조건]오직 JSON 문자열만 출력을 해라. 코드블록 금지. [출력양식]{fmt}"""),
    ("user", """다음 텍스트에서 사용자 정보를 추출해줘 [텍스트]{text}""")
]
).partial(fmt = fmt)

profile_chain = profile_prompt | model | profile_parser

In [223]:
def update_profile_from_text(session_id: str, text : str):
    profile = profile_chain.invoke({'text' : text})
    print(profile)
    set_profile(session_id, profile)

In [224]:
# 1. 프롬프트에 history 자리 확보
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 AI 도우미야, 간략하게 그냥 응답하도록 해"),
    ("system", "[사용자 정보] {profile}"),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{question}"),
])
prompt

ChatPromptTemplate(input_variables=['history', 'profile', 'question'], input_types={'history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotat

In [225]:
chain = prompt | model | StrOutputParser()

In [226]:
# 2. 대화 내용 저장소 만들기
stores : Dict[str, InMemoryChatMessageHistory] = {}
def get_store(session_id: str):
    print(f"[대화 세션ID]: {session_id}")
    if session_id not in stores:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        stores[session_id] = InMemoryChatMessageHistory()
    return stores[session_id]  # 해당 세션 ID에 대한 세션 기록 반환

In [227]:
def get_store(session_id: str):
    print(f"[대화 세션ID]: {session_id}")
    if session_id not in stores:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        stores[session_id] = InMemoryChatMessageHistory()
    return stores[session_id]  # 해당 세션 ID에 대한 세션 기록 반환

In [228]:
# 3. 히스토리랑 래핑
with_history = RunnableWithMessageHistory(
    chain,
    lambda sid: get_store(sid),
    input_messages_key="question",
    history_messages_key="history",
)

In [229]:
cfg = {"configurable" : {"session_id" : "user-1"}}

In [ ]:
def ask(question: str, session_id: str = 'user-1', conversation_id: str = 'conv-1'):
    """요약 갱신 -> 요약 텍스트를 입력에 포함해서 호출"""
    update_profile_from_text(session_id, question)
    config= {"configurable": {"session_id": session_id, "conversation_id": conversation_id}}
    return with_history.invoke({"question" : question, "profile" : get_profile(session_id)}, config=config)

In [231]:
ask('안녕하세요. 제 이름은 감자입니다')

name='감자' age=None gender=None tone='neutral'
[대화 세션ID]: user-1


'안녕하세요, 감자님! 어떻게 도와드릴까요?'

In [232]:
ask(' 제 나이는 30세 입니다')

name=None age=30 gender=None tone='neutral'
[대화 세션ID]: user-1


'네, 감자님. 30세이시군요. 도움이 필요하시면 말씀해 주세요.'

In [233]:
ask('저의 이름 나이 성별은 무엇입니까?')

name=None age=None gender=None tone='neutral'
[대화 세션ID]: user-1


'이름은 감자, 나이는 30세, 성별은 알려주지 않으셨습니다.'